In [691]:
from tabulate import tabulate
import pandas as pd
import pandas.io.common
import numpy as np
import os

In [692]:
characters=['\'',
            ',']

columns_to_delete=['energy',]
rows_to_delete=['kwh']

In [693]:
def dfToLowerCase(df):
    df = df.apply(lambda x: x.astype(str).str.lower())
    return df

In [694]:
def removeUnwantedCharacter(df,characters):
    for c in characters:
            for column in df:
                if df[column].dtype == object:
                    df[column] = df[column].replace({c:''}, regex=True)
    return df

In [695]:
def dropUnwantedColumns(df,columns_to_delete):
    print(tabulate(df, headers='keys', tablefmt='psql'))
    toDelete=[]
    columns = df.loc[:, df.columns != df.columns[0]].columns
    for column in columns:
        if df[column].dtype == object:
            if any(word in str(df[column].iloc[0]) for word in columns_to_delete):
                toDelete.append(column)
            if len(df[column])>1 and any(word in str(df[column].iloc[1]) for word in columns_to_delete):
                if column not in toDelete:
                    toDelete.append(column)
    for column in toDelete:
            df = df.drop(column, 1)
    return df
    

In [696]:
def searchForDate(df):
    print('searching for date')
    foundDate=''
    dates18=['2018','FY18','18/19']
    dates19=['2019','FY19','19/20']
    dates20=['2020','FY20','20/21']
    dates21=['2021','FY21','21/22']
    datesList=[dates18,dates19,dates20,dates21]
    print('collecting rows')
    try:
        rows=[df.loc[1].values.flatten().tolist(),
        df.loc[0].values.flatten().tolist()]
    except KeyError:#occurs when no second row for some columns
        rows=[df.loc[0].values.flatten().tolist()]
    print(rows)
    for row in rows:
        #print('Searching for row: ',row)
        for item in row:
            #print('item: ',item)
            for dates in datesList:
                #print('searching for date',dates)
                if any(date in item for date in dates):
                    #print('found date: ',dates[0])
                    foundDate = dates[0]

    for dates in datesList:
        if any(date in df.columns for date in dates):
            foundDate = dates[0]
    return foundDate



In [697]:
def getdatafromrow(df):
    print('getting data from row')
    print(tabulate(df, headers='keys', tablefmt='psql'))
    df = df.reset_index(drop=True)
    print(tabulate(df, headers='keys', tablefmt='psql'))
    values = df.loc[0].values.flatten().tolist()
    values.pop(0)
    data = ''
    #print(values)
    while data == '':
        #print('Checking items')
        for item in reversed(values):#could use reversed(values)

            if item !='nan':

                #print(item)
                numbers=[]
                numbers = [int(s) for s in item.split() if s.isdigit()]#extract integers from the string
                #print('Numbers: ',numbers)
                if numbers!=[]:
                    data = sorted(numbers, reverse=True)[0] #take the largest number found - to discount errors in the OCR process

                else:#if no integers are found, try and find floats
                    for token in item.split():
                        try:
                            #print (float(token))
                            numbers.append(float(token))
                        except ValueError:
                            pass
                    if numbers!=[]:
                        data = sorted(numbers, reverse=True)[0]
        break
    return data


In [698]:
number = '0.42'
for token in number.split():
    try:
        print (float(token))
    except ValueError:
        pass

0.42


In [699]:
a=[0,123]
x = sorted(a, reverse=True)
x

[123, 0]

In [700]:
def extractDataByPhrase(df,wantedPhrases,**kwargs):
    #print('extracting data by phrase')
    #print(tabulate(df, headers='keys', tablefmt='psql'))
    #print(wantedPhrases)
    unwantedPhrases = kwargs.get('unwantedPhrases', None)
    sumIfMultiple = kwargs.get('sumIfMultiple', True)
    returnData = ''
    '''
    Filters the Dataframe to df_f which only includes rows with the filter phrases defined & without unwanted phrases
    '''
    if len(wantedPhrases)==1:
        #print('Single wanted phrases detected')
        df_f = df[df[0].str.contains(wantedPhrases[0])]#filters to rows where the label contains the wanted phrase
    else:
        df_f = df[df[0].str.contains(wantedPhrases[0])]#search for first wanted phrase & create new dataframe
        for i in range(1, len(wantedPhrases)):#search for the rest of the phrases & concat the dataframe
            df_f=pd.concat([df_f,df[df[0].str.contains(wantedPhrases[i])]])
        #print('Finished Searching for multiple phrases')
        #print(tabulate(df_f, headers='keys', tablefmt='psql'))
    if unwantedPhrases is not None:
        #print('dropping unwanted phrases')
        for phrase in unwantedPhrases:
            df_f = df_f[~df_f[0].str.contains(phrase)]#'filters out the unwanted phrases
    df_f = df_f.reset_index(drop=True)#resets the index
    #print(tabulate(df_f, headers='keys', tablefmt='psql'))
    
    '''
    extracting data from the dfiltered dataframe
    if only 1 line, extracts that
    if multiple rows, extracts them all, and sums depending on argument given
    '''
    
    if len(df_f)==1:#if there is only 1 line of scope 1 data
        returnData = getdatafromrow(df_f)#extract the value from that row

    elif len(df_f)>1 and sumIfMultiple:#if multiple scope 1 values found

        #if there's a total scope 1 value, take that
        temp_df = df_f[df_f[0].str.contains("total")]
        if not temp_df.empty:
            returnData = getdatafromrow(temp_df)
            temp_df=[]

        #otherwise add all scope 1 values together
        else:
            values=[]#holds each scope 1 value
            for i in range(len(df_f)):#for each row
                print('getting data for row: '+str(i))
                #print('Scope 1 Value: '+scope1_df.filter(items = [i], axis=0))
                values.append(getdatafromrow(df_f.filter(items = [i], axis=0)))
            values = list(filter(None, values))
            print(values)
            returnData = sum(values)

    elif len(df_f)>1 and sumIfMultiple==False:
        #print('multiple phrases detected. No Sum argument given. giving first value in values columns.')
        returnData = getdatafromrow(df_f.head(1))
    #print(scope1)
    return returnData
            
'''
HERE
'''    


'\nHERE\n'

In [701]:
from numpy import genfromtxt
import csv

def handleParseError(folder,file):
   with open(folder+'\\'+file, 'r') as read_obj:
      csv_reader = csv.reader(read_obj)
      list_of_csv = list(csv_reader)
      columnLen=len(list_of_csv[0])
      
      print(columnLen)
      outputList=[]
      for row in list_of_csv:
         outputList.append(row[:columnLen])
      print(outputList)
      df=pd.DataFrame(outputList)
      return df

In [702]:
def dropUnwantedRows(df,filterPhrases):
    print('Deleting Unwanted Rows')
    columnsToDelete=[]
    '''
    Can use to drop columns with phrases like KWH we dont want to accidently collect data for
    '''

    #df = df[~df[0].str.contains(['kWh'])]
    df[~df[0].str.contains('Kwh')]
    return df


In [703]:
from pandas.io.parsers import ParserError
folder = r'C:\Users\jacks\Documents\Document(Offline)\Barcanet\Record Linkage\Filtered\test'

output=[]

for file in os.listdir(folder):
    try:
        if file.endswith(".csv"):
            print('file: '+file)
            try:
                df = pd.read_csv(folder+'\\'+file,encoding='iso-8859-1',header=None)
            except ParserError:
                df = handleParseError(folder,file)
                
            print('Original Dataframe ')
            print(tabulate(df, headers='keys', tablefmt='psql'))
            
            df = dfToLowerCase(df) #making df lower case
            df = removeUnwantedCharacter(df,characters)#remove unwanted characters from the whole of the dataframe
            df = dropUnwantedColumns(df,columns_to_delete)#drop columns if their header is unwanted e.g units
            #
            df = df.dropna(how='all', axis=1)#remove empty columns
            df = df.dropna(how='all')#remove empty rows
            df = dropUnwantedRows(df,rows_to_delete)
            print('Altered Dataframe')
            print(tabulate(df, headers='keys', tablefmt='psql'))
            if len(df.columns)>1:
                foundDate = searchForDate(df)
                
                scope1 = extractDataByPhrase(df,['scope 1'],unwantedPhrases=['scope 1+2',
                                                'scope 1 + 2',
                                                'scope 1 +2',
                                                'scope 1+ 2'])
                scope2 = extractDataByPhrase(df,['scope 2'])
                '''scope12 = extractDataByPhrase(df,['scope 1+2',
                                                'scope 1 + 2',
                                                'scope 1 +2',
                                                'scope 1+ 2'],
                                                sumIfMultiple=False)'''
                scope3 = extractDataByPhrase(df,['scope 3'])
                intensity = extractDataByPhrase(df,['intensity'])
                totalemissions= extractDataByPhrase(df,['total gross co2e',
                                                    'total gross emissions',
                                                    'total net emissions',
                                                    'total emissions',
                                                    'total co2e',
                                                    'total gross co?e',
                                                    'total co?e',
                                                    'all scopes'],sumIfMultiple=False)

                print('Date: '+foundDate)
                print('scope 1 value: ',scope1)
                print('scope 2 value: ',scope2)
                #print('scope 1 + 2 value: ',scope12)
                print('scope 3 value: ',scope3)
                print('Intensity: ', intensity)
                print('total emissions: ',totalemissions)
                output.append([file[:-4],foundDate,scope1,scope2,scope3,intensity,totalemissions])
    except pandas.io.common.EmptyDataError:
        print('Dataframe Empty.')
        
import numpy as np
df_output = pd.DataFrame(output,columns=['Company Number','Date','Scope 1','Scope 2','Scope 3','Intensity','Total Emissions'])
df_output.to_csv(folder+'\\output.csv',index=False)
df_output

file: 443687.csv
Original Dataframe 
+----+---------------------------------------------------------------------------------------+------------+
|    | 0                                                                                     | 1          |
|----+---------------------------------------------------------------------------------------+------------|
|  0 | Current reporting year                                                                | 2020       |
|  1 | Location                                                                              | nan        |
|  2 | Emissions from the combustion of gas and fue! in rental vehicles (Scope 1) (tCO2e     | 1,377      |
|  3 | Emissions from purchase of electricity (tCO2e) (Scope 2                               | nan        |
|  4 | Emissions from business travel in employee-owned and company vehicles where Adient    | 124        |
|    | Seating UK Ltd Is responsible for purchasing the fue! and emissions from rail and alr |     

,Company Number,Date,Scope 1,Scope 2,Scope 3,Intensity,Total Emissions
0,443687,2020,7434238,10154275,524746,0.002,4084
1,output,2020,,,,,
